In [37]:
from sklearn import svm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [20]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisosONEHOT.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantesONEHOT.csv")
vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")
postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [21]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
no_postulaciones=pd.merge(no_postulaciones,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
no_postulaciones=pd.merge(no_postulaciones,avisos,on='idaviso')

no_postulaciones.head()

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Por Horas,tipo_de_trabajo_Primer empleo,tipo_de_trabajo_Teletrabajo,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior
0,1112430536,EexaOz,0,38.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1112430536,owaWw0Z,0,20.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,1112430536,PmGPRd1,0,19.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,1112430536,bO4q324,0,25.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,1112430536,MVd5dlr,0,30.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [22]:
postulaciones=postulaciones.append(no_postulaciones, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)

postulaciones = postulaciones.sample(50000)
print(len(postulaciones))
no_postulaciones = 0

12793168
50000


In [23]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.total_vistas=postulaciones.total_vistas.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Teletrabajo,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,cantidad,total_vistas
35907,1112326604,8x1NKL,0,38.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
13346,1112359096,KBzkRKq,0,25.0,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,0.0,0.0
37934,1112350476,GNZ31Wv,1,29.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
38341,1112353872,1Q4WWLQ,0,29.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
2068,1112248716,ekq9aj,1,36.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
31526,1112424887,6rQrZZ3,1,26.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0.0,0.0
33045,1112414725,2zk0x4x,1,22.0,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,2.0,6.0
33243,1112299770,owjrjzZ,0,22.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
33557,1112462970,ak4jLvE,0,25.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0
48256,1111472606,W9WYN8m,0,28.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,0.0,0.0


In [24]:
areas=avisos['nombre_area'].value_counts().index
postulaciones.drop(["nombre_area"],axis=1,inplace=True)
#postulaciones['nombre_area']=postulaciones['nombre_area'].replace(areas, range(len(areas)))


In [25]:
train, test=train_test_split(postulaciones,test_size=0.10,random_state=int(time.time()))
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
labels=train['sepostulo']

x_test=test[features]
labels_test=test['sepostulo']


gc.collect()

Train:  45000 Test:  5000


70

In [32]:
def roc_score(model):
    x_final = x_test[features]
    y_final = model.predict(x_final)
    y_pred = []
    for y in y_final:
        y_pred.append((y[1]))
    return roc_auc_score(labels_test,y_pred)

In [43]:
model = svm.SVC()
model.fit(x_train,labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [44]:
score = model.score(x_test,labels_test)
print(score)

0.6964


In [45]:
x_final = x_test[features]
y_final = model.predict(x_final)
y_final

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [46]:
y_pred = []
for y in y_final:
    y_pred.append(y)
roc_auc_score(labels_test,y_pred)

0.6865055531171635